# Imports

In [5]:
import numpy as np
import pandas as pd
import nltk
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer 


# Reading data

In [2]:
df = pd.read_csv('dataset/trainning-tweets.csv',encoding = "ISO-8859-1",names=['sentiment','id','date','Query','user','tweet']) 

In [3]:
X = df.tweet.values.tolist()
y = df.sentiment.values.tolist()

# Cleanning

### Remove links

In [77]:
X_nolinks = [re.sub(r'http\S+', '', sentence) for sentence in X]
X_nolinks = [re.sub(r'https\S+', '', sentence) for sentence in X_nolinks]
X_nolinks = [re.sub(r'www\S+', '', sentence) for sentence in X_nolinks]

### Remove sitações do twitter como @user

In [78]:
X_noRef = [re.sub(r'@\S+', '', sentence) for sentence in X_nolinks]

### Remove caracteres especiais 

In [66]:
X_new = [re.sub(r"[^a-zA-Z0-9]+", ' ', sentence.lower()) for sentence in X_noRef]

In [76]:
for i in range(20000,20010):
    print(X[i])
    #print(X_nolinks[i])
    #print(X_noRef[i])
    print(X_new[i])
    #print(X_new2[i])

@heather2711 Good thing I didn't find any then  None of the ones I like come in my size. Stupid big feet 
 good thing i didn t find any then none of the ones i like come in my size stupid big feet 
DEA's are no fun  
dea s are no fun 
@tommcfly no i haven't  hey but you guys are back in England!!! Welll it was super great last night!! Can't wait for the next gig!
 no i haven t hey but you guys are back in england welll it was super great last night can t wait for the next gig 
I googled up remedies for jonzing and reading was one of the suggestions. therefore i'm now reading angels and demons.. it's pretty bunk 
i googled up remedies for jonzing and reading was one of the suggestions therefore i m now reading angels and demons it s pretty bunk 
trying to fix my phone         fail
trying to fix my phone fail
@amazingphoebe mum said you couldn't come over 
 mum said you couldn t come over 
oh well back to work tomorrow 
oh well back to work tomorrow 
Powerpoint on a Sunday morning 
power

# Stemming with PorterStemmer

In [10]:
snowball=SnowballStemmer(language="english")

In [11]:
# def stemSentence(sentence):
#     token_words=word_tokenize(sentence)
#     token_words
#     stem_sentence=[]
#     for word in token_words:
#         stem_sentence.append(snowball.stem(word))
#         stem_sentence.append(" ")
#     return "".join(stem_sentence)

In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
# def lemmSentence(sentence):
#     token_words=word_tokenize(sentence)
#     token_words
#     stem_sentence=[]
#     for word in token_words:
#         stem_sentence.append(lemmatizer.lemmatize(word))
#         stem_sentence.append(" ")
#     return "".join(stem_sentence)

# Próxima linha demora ~10 mins

In [14]:
# X_new = []
# for sentence in tqdm(X):
#     stem_sentence = stemSentence(sentence)
#     new_sentence = lemmSentence(sentence)
#     X_new.append(new_sentence)

# Caracterização dos textos com TFIDF e remoção de stopwords

In [68]:
vect = TfidfVectorizer(stop_words='english')

In [72]:
mnb = MultinomialNB(alpha=0.5)
# log_model = LogisticRegression()
# stemmer = nltk.stem.RSLPStemmer()


In [73]:
pipe = Pipeline(steps=[('vect', vect)
                       ,('mnb', mnb)
                      ])
param_grid = { 
}

In [83]:
search = GridSearchCV(pipe, param_grid, iid=False, cv=5, return_train_score=False)
search.fit(X_noRef, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('mnb', MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=False, n_jobs=None, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [84]:
result_df = pd.DataFrame(search.cv_results_)
result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,15.568531,0.916648,3.650883,0.283175,{},0.752059,0.746534,0.745162,0.755484,0.749356,0.749719,0.003735,1


In [88]:
search.predict(['Dude, i''m so sad and happy right now!!!'])

array([0])